# MultiLayeredNetwork Tutorial

<span style="font-size:13pt">In this tutorial we show how the MultiLayeredNetwork (MLN) class can be used to work with the dummy POPNET network. For that we go through the relevant methods and show simple examples of how to use them. We show most functionalities on a subgraph of the network to keep processing times lower, but since this is also an MLN class, everything of course also works on the entire network.</span>

<span style="font-size:13pt">To import the class:</span>

In [1]:
# might need to be altered depending on where mln.py is located
import sys
sys.path.append("../src")
from mln import MultiLayeredNetwork

<br><span style="font-size:13pt">To create an instance of the MLN class using all POPNET data:</span>

In [2]:
popnet = MultiLayeredNetwork(
    adjacency_file="../test_data/dummy_popnet/popnet_edgelist.csv",
    node_attribute_file="../test_data/dummy_popnet/popnet_nodelist.csv"
)

In [3]:
popnet.A

<146x146 sparse matrix of type '<class 'numpy.int64'>'
	with 3680 stored elements in Compressed Sparse Row format>

In [4]:
popnet.node_attributes

,label,generation,gender,NID
0,50,1,0,0
1,51,1,1,1
2,52,1,1,2
3,53,1,1,3
4,54,1,1,4
...,...,...,...,...
141,195,3,0,141
142,196,3,0,142
143,197,3,1,143
144,198,3,0,144


---

## Filtering

<br><span style="font-size:13pt">To get an instance of the MLN class only containing **nodes with certain labels**</span>

In [5]:
from random import sample
# selecting females
filtered_ids = popnet.node_attributes[popnet.node_attributes["gender"]==0]["label"]

In [6]:
filtered = popnet.get_filtered_network(selected_nodes=filtered_ids)

print("Number of people:", filtered.N)
print("Number of connected pairs of people:", filtered.A.nnz)

Number of people: 76
Number of connected pairs of people: 914


<br><span style="font-size:13pt">To get an instance of the MLN class containing just **certain layers**, e.g. the household layer:</span>

In [7]:
filtered_h = filtered.get_filtered_network(layers=["household"])

print("Number of people:", filtered_h.N)
print("Number of connected pairs of people:", filtered_h.A.nnz)

Binary repr 32
Number of people: 76
Number of connected pairs of people: 50


<br><span style="font-size:13pt">These filtering methods can also be used in **any combination**, e.g. all together:</span>

In [10]:
filtered = popnet.get_filtered_network(
    selected_nodes=filtered_ids,
    layers=["household", "work"]
)

print("Number of people:", filtered.N)
print("Number of connected pairs of people:", filtered.A.nnz)

Binary repr 33
Number of people: 76
Number of connected pairs of people: 180


---

## Conversion to other formats

### igraph

<span style="font-size:13pt">Note that igraph automatically labels nodes from 0 through N-1 but these labels very likely do not line up with the mapping from IDs to labels as stored in `mln.nodemap_back`. However, we do always store the labels as node attributes.</span>

In [11]:
# prerequisites, also make sure to import the MLN class and create the
# `popnet` instance, as shown at the very start of this tutorial

import igraph as ig

filtered = popnet.get_filtered_network(selected_nodes=filtered_ids)

---

<span style="font-size:13pt">To obtain an **igraph Graph object** representing the MLN:</span>

In [12]:
g_igraph = filtered.to_igraph()

ig.summary(g_igraph) # print the node count, edge count, and a list of the available attributes

IGRAPH D--- 76 914 -- 
+ attr: label (v), link_types (e)


<br><span style="font-size:13pt">To obtain an **undirected** igraph Graph object of the MLN:</span>

In [13]:
g_igraph_u = filtered.to_igraph(directed=False)

ig.summary(g_igraph_u)
print("Directed:", g_igraph_u.is_directed())

IGRAPH U--- 76 457 -- 
+ attr: label (v), link_types (e)
Directed: False


<br><span style="font-size:13pt">To **omit link types and node attributes** (except for the labels):</span>

In [14]:
g_igraph = filtered.to_igraph(edge_attributes=False, node_attributes=False)

ig.summary(g_igraph)

IGRAPH D--- 76 914 -- 
+ attr: label (v), link_types (e)


<br><span style="font-size:13pt">The igraph objects are also stored in the `igraph` attribute of the MLN class. Unless we specify otherwise, only the result from the first call to `.get_igraph()` will be stored. To overwrite this:</span>

In [15]:
print("Before:")
ig.summary(filtered.igraph)

# overwrite using replace_igraph=True
filtered.to_igraph(replace_igraph=True, directed=False, node_attributes=False)

print("\nAfter:")
ig.summary(filtered.igraph)

Before:
IGRAPH D--- 76 914 -- 
+ attr: label (v), link_types (e)

After:
IGRAPH U--- 76 457 -- 
+ attr: label (v), link_types (e)


<br>

---

### NetworkX

<span style="font-size:13pt">Note that NetworkX automatically labels nodes from 0 through N-1 but these labels very likely do not line up with the mapping from IDs to labels as stored in the class. However, we do always store the labels as node attributes.</span>

In [16]:
# prerequisites, also make sure to import the MLN class and create the
# `popnet` instance, as shown at the very start of this tutorial

import networkx as nx

filtered = popnet.get_filtered_network(selected_nodes=filtered_ids)

---

<span style="font-size:13pt">We can also obtain a **NetworkX graph** representation of the MLN. Since NetworkX is a less efficient library, we recommend to only use if for networks smaller than `mln.nx_limit`. To create a NetworkX graph for a network that is larger:</span>

In [17]:
g_nx = filtered.to_networkx(ignore_limit=True)

print(nx.info(g_nx))

DiGraph with 76 nodes and 914 edges


<br><span style="font-size:13pt">To obtain an **undirected** version of the network:</span>

In [18]:
g_nx = filtered.to_networkx(node_attributes=True,directed=False, ignore_limit=True)

print(nx.info(g_nx))

Graph with 76 nodes and 457 edges


<br><span style="font-size:13pt">To **node attributes** (except for the labels):</span>

In [19]:
print(g_nx.nodes[0])
print(g_nx.edges[0, 15]) # inspect one link manually to show the link type

# new graph without node and edge attributes
g_nx = filtered.to_networkx(edge_attributes=False, node_attributes=False, ignore_limit=True)

print()
print(g_nx.nodes[0])
print(g_nx.edges[0,15])

{'label': 0, 'generation': 1, 'gender': 0, 'NID': 0}
{'link_types': ['sister / brother-in-law']}

{'label': 0}
{}


<span style="font-size:13pt">The NetworkX networks are not stored directly as an attribute of the MLN class, and so there is no replace function as there is for igraph.</span>

---


## Exporting and importing

In [20]:
# prerequisites, also make sure to import the MLN class and create the
# `popnet` instance, as shown at the very start of this tutorial

import pandas as pd

filtered = popnet.get_filtered_network(selected_nodes=filtered_ids)

In [21]:
import os
# setting current path as library directory
filtered.library_path = os.getcwd()

---

<span style="font-size:13pt">To **export all data** in an MLN object to a library:</span>

In [22]:
filtered.export(library_name="filtered_full")

Library "/home/bokanyie/projects/popnet_mln/tutorial/filtered_full" already exists, call function with `overwrite = True` keyword argument if you're sure!


<span style="font-size:13pt">If a folder with the given name already exists, we can overwrite it with the argument `overwrite=True`.</span>

<br><span style="font-size:13pt">To **import** from a libary:</span>

In [23]:
filtered2 = MultiLayeredNetwork(
    from_library="filtered_full",
    library_path=filtered.library_path
)

<br><span style="font-size:13pt">The **node attributes** can also be exported separately:</span>

In [24]:
filtered.export_node_attributes("filtered_node_attrs.csv.gz")

<span style="font-size:13pt">Uncompressed files and other file separators are also available.</span>

<br><span style="font-size:13pt">The **adjacency matrix** can also be exported separately:</span>

In [25]:
filtered.export_graph("filtered_graph.graphml")

<span style="font-size:13pt">Other file types and separators are also available.</span>

---

## Other functionalities / examples

In [30]:
# prerequisites, also make sure to import the MLN class and create the
# `popnet` instance, as shown at the very start of this tutorial

---

<span style="font-size:13pt">To obtain the ego network of a person at a certain depth (the returned object is also of the MLN class):</span>

In [32]:
ego = popnet.get_egonetwork(popnet.map_nid_to_label[0], depth=2)

<br><span style="font-size:13pt">To create an affiliation matrix between people and a certain attribute, e.g. gender:</span>

In [33]:
# first create an edgelist of (person, attribute value) pairs 
affiliation_edgelist = popnet.node_attributes.set_index("label")['gender'].dropna().to_dict().items()

# create the affiliation matrix, under key 'work'
popnet.create_affiliation_matrix('gender', affiliation_edgelist)

<span style="font-size:13pt">This affiliation matrix is now stored in `.affiliation_matrix`, which is a dictionary that can store several affiliation matrices. We can acces the one that was just made with key "work" using:</span>

In [36]:
popnet.affiliation_matrix["gender"]["A"]

<146x2 sparse matrix of type '<class 'numpy.int64'>'
	with 146 stored elements in Compressed Sparse Row format>